In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import s3fs
import pandas as pd

os.chdir("../")

from src.core.configs import settings, fs
from src.dataloader.loader import CustomImageDataset, MaskDataloaders
from src.model.base_model import BaseModel
from src.model.net_model import Net


In [ ]:
train, val= MaskDataloaders(settings.s3_prefix, 0.7).getDataloaders()

In [ ]:
net = Net(2)
model = BaseModel(net)
model.load_model(path='./checkpoint/loss_validation0.042.save')
# model.fit([train, val])